In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.join(os.path.dirname(os.getcwd())))

# Now you can use absolute imports
from src.utils.data_processing import load_data, preprocess_data
from src.utils.defaults import DEFAULTS, DIAMETER_DICT, THICKNESS_DICT, FLOWRATE_DICT, TEMPERATURE_DICT, PRESSURE_DICT

# User Specified Steady-State Regions

In [2]:
steady_state_regions = {
    'RUN_H_25C-50bar': [
        {
            "notes": "First steady state",
            "time_range": [40e3, 50e3],
        },
    ],
    
    'RUN_H_25C-100bar_7': [
        {
            "notes": "First steady state",
            "time_range": [29e3, 30e3],
        },        
    ],
    
    'RUN_H_25C-100bar_8': [
        {
            "notes": "First steady state",
            "time_range": [35e3, 40e3],
        },        
    ],
    
    'RUN_H_25C-100bar_9': [
        {
            "notes": "First steady state",
            "time_range": [35e3, 40e3],
        },        
    ],
    
    'RUN_H_25C-200bar_2': [
        {
            "notes": "First steady state",
            "time_range": [39e3, 40e3],
        },
    ],
    
    'RUN_H_50C-50bar': [
        {
            "notes": "First steady state",
            "time_range": [29e3, 30e3],
        },
    ],
    
    'RUN_H_50C-100bar_2': [
        {
            "notes": "First steady state",
            "time_range": [29e3, 30e3],
        },
    ],
    
    'RUN_H_50C-200bar': [
        {
            "notes": "First steady state",
            "time_range": [29e3, 30e3],
        },
    ],
    
    'RUN_H_75C-50bar': [
        {
            "notes": "First steady state",
            "time_range": [19e3, 20e3],
        },
    ],
    
    'RUN_H_75C-100bar': [
        {
            "notes": "First steady state",
            "time_range": [9.5e3, 10e3],
        },
    ],
    
    "S4R3": [
        # First, 25 bar
        {
            "notes": "First steady state",
            "time_range": [170e3, 220e3],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        },
        # Second, 50 bar
        {
            "notes": "Second steady state",
            "time_range": [880e3, 900e3],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        },
        # Third, 75 bar
        {
            "notes": "Third steady state",
            "time_range": [1.3e6, 1.4e6],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        },
        # Fourth, 100 bar
        {
            "notes": "Fourth steady state",
            "time_range": [2.1e6, 2.15e6],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        },
        # Fifth, 150 bar
        {
            "notes": "Fifth steady state",
            "time_range": [2.7e6, 2.75e6],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        },
        # Sixth, 200 bar
        {
            "notes": "Sixth steady state",
            "time_range": [2.82e6, 2.85e6],
            # "pressure": "Unknown",  # Average will be calculated
            # "flux": "Unknown",  # Average  will be calculated
        }
    ],
    
    'S4R4': [
        # First, 50 bar
        {
            "notes": "First steady state",
            "time_range": [40e3, 50e3],
        },
        # Second, 100 bar
        {
            "notes": "Second steady state",
            "time_range": [250e3, 260e3],
        },
        # Third, 150 bar
        {
            "notes": "Third steady state",
            "time_range": [340e3, 350e3],
        },
        # Fourth, 200 bar
        {
            "notes": "Fourth steady state",
            "time_range": [530e3, 540e3],
        },
        # Fifth, 150 bar
        {
            "notes": "Fifth steady state",
            "time_range": [630e3, 640e3],
        },
    ],
    
    'S4R5': [
        # First, 50 bar
        {
            "notes": "First steady state",
            "time_range": [60e3, 80e3],
        },
        # Second, 100 bar
        {
            "notes": "Second steady state",
            "time_range": [140e3, 160e3],
        },
        # Third, 150 bar
        {
            "notes": "Third steady state",
            "time_range": [220e3, 240e3],
        },
        # Fourth, 200 bar
        {
            "notes": "Fourth steady state",
            "time_range": [380e3, 450e3],
        },
        # Fifth, 150 bar
        {
            "notes": "Fifth steady state",
            "time_range": [620e3, 630e3],
        },
        # Sixth, 100 bar
        {
            "notes": "Sixth steady state",
            "time_range": [710e3, 720e3],
        },
        # Seventh, 50 bar
        {
            "notes": "Seventh steady state",
            "time_range": [800e3, 810e3],
        },
    ],
    
    'S4R6': [
        # First, 50 bar
        {
            "notes": "First steady state",
            "time_range": [50e3, 70e3],
        },
        # Second, 100 bar
        {
            "notes": "Second steady state",
            "time_range": [150e3, 160e3],
        },
        # Third, 150 bar
        {
            "notes": "Third steady state",
            "time_range": [200e3, 210e3],
        },
        # Fourth, 200 bar
        {
            "notes": "Fourth steady state",
            "time_range": [290e3, 300e3],
        },
    ]
}

In [3]:
# Calculate missing pressures from data
def calculate_missing_pressures_fluxes(regions_dict, df, experiment_name):
    """Calculate average pressure for regions where pressure is not specified"""
    if experiment_name in regions_dict:
        for region in regions_dict[experiment_name]:
            # If pressure is not specified or "Unknown", calculate it from the data
            if 'pressure' not in region or region['pressure'] == "Unknown":
                time_start, time_end = region['time_range']
                # Filter data within time range
                mask = (df['time'] >= time_start) & (df['time'] <= time_end)
                region_data = df[mask]
                
                if len(region_data) > 0:
                    region['pressure'] = region_data['pressure'].mean()
                else:
                    region['pressure'] = None  # No data in this time range
            
            # if flux is not specified or "Unknown", calculate it from the data
            if 'flux' not in region or region['flux'] == "Unknown":
                time_start, time_end = region['time_range']
                # Filter data within time range
                mask = (df['time'] >= time_start) & (df['time'] <= time_end)
                region_data = df[mask]
                
                if len(region_data) > 0:
                    region['flux'] = region_data['flux'].mean()
                else:
                    region['flux'] = None
    return regions_dict

In [4]:
import pandas as pd
from pathlib import Path

# Create list to store all results
results = []

print("Processing steady state data for all experiments:")
for experiment, regions in steady_state_regions.items():
    print(f"\n{experiment}:")
    
    # Read data
    if 'S4' in experiment:
        # Use multi-pressure path for S4
        data_path = Path.cwd().parent / 'data' / 'multi_pressure' / f'{experiment}.xlsx'
    else:
        # Use single-pressure path for the rest                
        data_path = Path.cwd().parent / 'data' / 'single_pressure' / f'{experiment}.xlsx'
    
    # Read and preprocess data
    exp_data = pd.read_excel(data_path)
    processed_data = preprocess_data(
        data=exp_data, 
        temperature= TEMPERATURE_DICT[experiment],
        thickness= THICKNESS_DICT[experiment], 
        flowrate= FLOWRATE_DICT[experiment],
        diameter= DIAMETER_DICT[experiment])
    
    # Fill in missing pressures and fluxes
    complete_exp_dict = calculate_missing_pressures_fluxes(steady_state_regions, processed_data, experiment)
    
    # Extract data for each region
    for i, region in enumerate(regions, 1):
        flux_ss = region['flux']
        pressure = region['pressure']
        L = THICKNESS_DICT[experiment]
        temperature = TEMPERATURE_DICT[experiment]
        permeability = flux_ss * L / pressure  # [cm^3(STP) bar^-1 cm^-1 s^-1]
        
        # Add to results list
        results.append({
            'exp': experiment,
            'temperature / C': temperature,
            '_pressure / bar': PRESSURE_DICT[experiment],
            'pressure / bar': pressure,
            'thickness / cm': L,
            'steady_state_flux / cm^3(STP) cm^-2 s^-1': flux_ss,
            'permeability / cm^3(STP) bar^-1 cm^-1 s^-1': permeability
        })
        
        print(f"  Region {i}: p = {pressure:.0f}, permeability = {permeability:.3g} cm^3(STP) bar^-1 cm^-1 s^-1")

# Create DataFrame and export to CSV
results_df = pd.DataFrame(results)
output_path = Path.cwd() / 'steady_state_results.csv'
results_df.to_csv(output_path, index=False)

print(f"\nResults exported to: {output_path}")
print(f"Total number of steady state regions: {len(results_df)}")

# Display 
print(results_df)

Processing steady state data for all experiments:

RUN_H_25C-50bar:
  Region 1: p = 50, permeability = 4.82e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_25C-100bar_7:
  Region 1: p = 99, permeability = 3.12e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_25C-100bar_8:
  Region 1: p = 99, permeability = 3.03e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_25C-100bar_9:
  Region 1: p = 99, permeability = 3.03e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_25C-200bar_2:
  Region 1: p = 199, permeability = 1.54e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_50C-50bar:
  Region 1: p = 50, permeability = 9.69e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_50C-100bar_2:
  Region 1: p = 99, permeability = 8.75e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_50C-200bar:
  Region 1: p = 199, permeability = 4.86e-08 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_75C-50bar:
  Region 1: p = 50, permeability = 2.12e-07 cm^3(STP) bar^-1 cm^-1 s^-1

RUN_H_75C-100bar:
  Region 1: p = 99, permeability = 2.04e-07 cm^3(STP) bar^-1 cm^-1 s^-1

S4R3:
  Region 1: p = 24, perm